<h2> Naive Bayes Spam Filter </h2>
<p>We're going to compute: P(Spam| Words = words) = P(W = w| S) / [P(W = w| S) + P(W = w|~S)] <br />

Notice this implementation of Bayes' Rule includes the prior that P(Spam) = P(~Spam) = .5 <br />

Actual computations will be performed with log probabilities to avoid underflow</p>

In [5]:
#imports
%matplotlib inline
from __future__ import division
import re
from matplotlib import pyplot as plt
from collections import defaultdict
import math

In [2]:
def tokenize(message):
    message = message.lower()                      # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)  # extract the words
    return set(all_words)                          # remove duplicates (in np use np.unique)

In [3]:
def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])           # when looking up a key, it first creates a value [0,0]
    for message, is_spam in training_set:          # look through all the training set data of labeled emails
        for word in tokenize(message):             # look at each unique word in the tokenized version of the email
            counts[word][0 if is_spam else 1] += 1 # if in spam email, {word: [x + 1, y]} if not {word: [x, y + 1]}
    return counts

In [4]:
# creating estimated probabilities with pseudocount smoothing for words not in training set
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets (word, P(word|spam), P(word|-spam))"""
    return [(w,                                           # word
            (spam + k) / (total_spams + 2 * k),           # # of spam emails with that word + k / # of spam emails + 2k
            (non_spam + k) / (total_non_spams + 2 * k))   # smoothed proportion of all non spam emails with that word
           for w, (spam, non_spam) in counts.iteritems()] # do this for each word, thus returning a list of 3-tuples

In [6]:
# assign spam probabilities to messages
def spam_probability(word_probs, message):              # notice this f'n evaluates a single message
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    # iterate through each word in our vocabulary
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        
        # if *word* from training set vocab appears in the message add the log probability of seeing it
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
            
        # if *word* doesn't appear in the message add the log probability of _not_ seeing it
        # which is log(1 - probability of seeing it)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
            
    prob_if_spam = math.exp(log_prob_if_spam) # this is P(W = w| S) from RHS of Bayes Formula
    prob_if_not_spam = math.exp(log_prob_if_not_spam) # this is P(W = w|~S)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [7]:
class NaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
        
    def train(self, training_set):
        
        # count spam and non-spam messages 
        num_spams = len([is_spam
                        for message, is_spam in training_set
                        if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        # run training data through our "pipeline"
        word_counts = count_words(training_set) # {word: [spam_count, non_spam_count], ...}
        self.word_probs = word_probabilities(word_counts,
                                             num_spams,
                                             num_non_spams,
                                             self.k)
    def classify(self, message):
        return spam_probability(self.word_probs, message)